## ML Pipelines using SparkML


## Objectives

In this Jupyter notebook, I learned to:

 - Create a machine learning pipeline.
 - Add stages to the pipeline.
 - Run the pipeline.
 - Create a machine learning pipeline for regression.
 - Create a machine learning pipeline for classification.


In [1]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

### Importing Required Libraries

In [2]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

#import functions/Classes for sparkml

from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import StringIndexer

from pyspark.sql import SparkSession


# import functions/Classes for pipeline creation

from pyspark.ml import Pipeline

# import functions/Classes for metrics
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


# Examples


## Task 1 - Load data set


Create SparkSession


In [3]:
spark = SparkSession.builder.appName("ML Pipeline Example").getOrCreate()

24/07/20 14:04:10 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Download the data file


In [4]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/mpg.csv


--2024-07-20 14:04:15--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/mpg.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13891 (14K) [text/csv]
Saving to: ‘mpg.csv.6’

mpg.csv.6           100%[===================>]  13.57K  --.-KB/s    in 0s      

2024-07-20 14:04:15 (45.5 MB/s) - ‘mpg.csv.6’ saved [13891/13891]



Load the dataset into the spark dataframe


In [5]:
# Load mpg dataset
mpg_data = spark.read.csv("mpg.csv", header=True, inferSchema=True)

Print the schema of the dataset


In [6]:
mpg_data.printSchema()

root
 |-- MPG: double (nullable = true)
 |-- Cylinders: integer (nullable = true)
 |-- Engine Disp: double (nullable = true)
 |-- Horsepower: integer (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Accelerate: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Origin: string (nullable = true)



Show 5 rows from the dataset


In [7]:
mpg_data.show(5)

+----+---------+-----------+----------+------+----------+----+--------+
| MPG|Cylinders|Engine Disp|Horsepower|Weight|Accelerate|Year|  Origin|
+----+---------+-----------+----------+------+----------+----+--------+
|15.0|        8|      390.0|       190|  3850|       8.5|  70|American|
|21.0|        6|      199.0|        90|  2648|      15.0|  70|American|
|18.0|        6|      199.0|        97|  2774|      15.5|  70|American|
|16.0|        8|      304.0|       150|  3433|      12.0|  70|American|
|14.0|        8|      455.0|       225|  3086|      10.0|  70|American|
+----+---------+-----------+----------+------+----------+----+--------+
only showing top 5 rows



## Task 2 - Define pipeline stages


In [8]:
# Stage 1 - assemble the input columns into a single vector 
vectorAssembler = VectorAssembler(inputCols=["Weight", "Horsepower", "Engine Disp"], outputCol="features")
# Stage 2 - scale the features using standard scaler
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
# Stage 3 - create a linear regression instance
lr = LinearRegression(featuresCol="scaledFeatures", labelCol="MPG")

## Task 3 - Build the pipeline


In [9]:
# Build the pipeline
pipeline = Pipeline(stages=[vectorAssembler, scaler, lr])

## Task 4 - Split the data


In [10]:
# Split the data into training and testing sets
(trainingData, testData) = mpg_data.randomSplit([0.7, 0.3], seed=42)

## Task 5 - Fit the pipeline


In [11]:
# Fit the pipeline to the training data
model = pipeline.fit(trainingData)

24/07/20 14:06:46 WARN util.Instrumentation: [318e44fd] regParam is zero, which might cause numerical instability and overfitting.
24/07/20 14:06:46 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/07/20 14:06:46 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
24/07/20 14:06:46 WARN netlib.LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
24/07/20 14:06:46 WARN netlib.LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


## Task 6 - Evaluate the model


Make predictions on the testing data


In [12]:
predictions = model.transform(testData)

Print the rmse value


In [13]:
evaluator = RegressionEvaluator(labelCol="MPG", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) =", rmse)

Root Mean Squared Error (RMSE) = 4.69281610922322


Stop Spark Session


In [14]:
spark.stop()

# Exercises


### Exercise 1 - Load data set


Create SparkSession with appname "ML Pipeline Exercise"


In [16]:
spark = SparkSession.builder.appName("ML Pipeline Exercise").getOrCreate() #TODO

Download the iris data set


In [17]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/iris.csv


--2024-07-20 14:08:55--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/iris.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4612 (4.5K) [text/csv]
Saving to: ‘iris.csv.1’

iris.csv.1          100%[===================>]   4.50K  --.-KB/s    in 0.002s  

2024-07-20 14:08:55 (2.64 MB/s) - ‘iris.csv.1’ saved [4612/4612]



Load the dataset into the spark dataframe


In [20]:
iris_data = spark.read.csv("iris.csv",header=True,inferSchema=True)

Print the schema of the dataset


In [21]:
iris_data.printSchema()

root
 |-- SepalLengthCm: double (nullable = true)
 |-- SepalWidthCm: double (nullable = true)
 |-- PetalLengthCm: double (nullable = true)
 |-- PetalWidthCm: double (nullable = true)
 |-- Species: string (nullable = true)



Show 5 rows from the dataset


In [22]:
iris_data.show(5)

+-------------+------------+-------------+------------+-----------+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+-------------+------------+-------------+------------+-----------+
|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
+-------------+------------+-------------+------------+-----------+
only showing top 5 rows



### Exercise 2 - Define pipeline stages


Stage 1 - Create an indexer stage using StringIndexer that will convert the Species column into a numeric column named "label"


In [23]:
indexer = StringIndexer(inputCol="Species",outputCol="label")

Stage 2 - Create a vectorAssembler stage that creates a feature vector named features using "SepalLengthCm", "SepalWidthCm", "PetalLengthCm","PetalWidthCm"


In [24]:
vectorAssembler = VectorAssembler(inputCols=["SepalLengthCm", "SepalWidthCm", "PetalLengthCm","PetalWidthCm"], outputCol="features")

Stage 3 - Create a scaler stage that scales the features using standard scaler, name the output columns as scaledFeatures


In [25]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

Stage 4 - Create a logistic regression stage using featuresCol="scaledFeatures", labelCol="label"


In [26]:
classifier = LogisticRegression(featuresCol="scaledFeatures", labelCol="label")

### Exercise 3 - Build the pipeline


Build a pipeline with all the four stages created earlier. 


In [27]:
pipeline = Pipeline(stages=[indexer,vectorAssembler, scaler, classifier])

### Exercise 4 - Split the data


Split the data into training and testing sets


In [28]:
(trainingData, testData) = iris_data.randomSplit([0.7, 0.3], seed=42)

### Exercise 5 - Fit the pipeline


Fit the pipeline to the training data


In [29]:
model = pipeline.fit(trainingData)

### Exercise 6 - Evaluate the model


Make predictions on the testing data


In [31]:
predictions = model.transform(testData)

# Evaluate model performance


Print the RMSE value


In [32]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy =", accuracy)

Accuracy = 0.9722222222222222


Stop Spark Session


In [33]:
spark.stop()